In [1]:
import json
from datasets import load_dataset, Dataset

# Load your dataset from a local JSON file (adjust paths as needed)
data_files = {
    "train": "/home/surenoobster/Documents/controllable-readability-summarization/src/18dec_retried_finalboss/train_score_category.json",
}

# Load dataset from local files
dataset = load_dataset("json", data_files=data_files)

# Function to format dataset for fine-tuning (using `input` as input text and `summary` as the output)
def format_for_finetuning(dataset):
    formatted_data = []
    for entry in dataset["train"]:  # Access the 'train' split
        # Use 'input' as the input text
        input_text = entry["input"]
        output_text = entry["summary"]  # Use 'summary' as the target output
        
        # Add formatted entry to the list
        formatted_data.append({
            "input": input_text,
            "output": output_text
        })
    return formatted_data

# Format the dataset
formatted_dataset = format_for_finetuning(dataset)

# Convert the formatted data into a Hugging Face Dataset for further processing
formatted_dataset = Dataset.from_list(formatted_dataset)

# Save the formatted dataset to a JSON file (optional)
output_file = "/home/surenoobster/Documents/controllable-readability-summarization/src/18dec_retried_finalboss/read_for_scoreFREfinetune_flanT5.json"
formatted_dataset.to_json(output_file)

print(f"Formatted dataset saved to: {output_file}")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/surenoobster/.cache/huggingface/datasets/json/default-9a3e761403c86782/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Formatted dataset saved to: /home/surenoobster/Documents/controllable-readability-summarization/src/18dec_retried_finalboss/read_for_scoreFREfinetune_flanT5.json


In [2]:
from datasets import load_dataset

# Path to your dataset
train_data_path = "/home/surenoobster/Documents/controllable-readability-summarization/src/18dec_retried_finalboss/read_for_scoreFREfinetune_flanT5.json"

# Load dataset from the local file
custom_dataset = load_dataset("json", data_files={"train": train_data_path})

# Check dataset sizes
print(f"Train dataset size: {len(custom_dataset['train'])}")

# Preview a random sample
from random import randrange

sample = custom_dataset['train'][randrange(len(custom_dataset['train']))]
print(f"Input (Prompt + Article): \n{sample['input']}\n---------------")
print(f"Output (Summary): \n{sample['output']}\n---------------")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/surenoobster/.cache/huggingface/datasets/json/default-2fa289790e7b13c7/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Train dataset size: 4884
Input (Prompt + Article): 
Write highlights for this article with a Flesch-Kincaid score of 45:

By . Daily Mail Reporter . and Ap . The woman featured in a photograph that shocked the world last year - that shows her giving birth on the lawn of a medical clinic after she was refused treatment by health officials - has made a full recovery and is in perfect health, as is her five-month-old son, Sabino. Additionally, the attention her shocking photo received has inspired women's rights activists across the globe to end what they call a pattern of poor . indigenous Mexican women being turned away from hospitals while in . labor, forcing them to give birth on lawns, patios or parking lots. The shocking image, taken in October by a passerby, . shows 29-year-old Irma Lopez , who is of Mazatec ethnicity, squatting . after giving birth, her face contorted in pain and her tiny newborn son . still bound by the umbilical cord and lying on the ground. Healthy: Irma Lopez 

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import concatenate_datasets, load_dataset

# Model ID for FLAN-T5
model_id = "google/flan-t5-small"

# Load tokenizer for FLAN-T5
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Path to the formatted dataset
train_path = "/home/surenoobster/Documents/controllable-readability-summarization/src/18dec_retried_finalboss/read_for_scoreFREfinetune_flanT5.json"

# Load the formatted dataset
data_files = {"train": train_path}
print("Loading dataset...")
dataset = load_dataset("json", data_files=data_files)

# Compute maximum input and output lengths for efficient batching
print("Calculating max input and target lengths...")

# Tokenize the concatenated dataset for inputs
tokenized_inputs = dataset["train"].map(
    lambda x: tokenizer(x["input"], truncation=True),
    batched=True,
    remove_columns=["input", "output"]
)
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# Tokenize the concatenated dataset for targets
tokenized_targets = dataset["train"].map(
    lambda x: tokenizer(x["output"], truncation=True),
    batched=True,
    remove_columns=["input", "output"]
)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

# Preprocessing function
def preprocess_function(sample, padding="max_length"):
    # Prepare inputs with a task prefix (e.g., "summarize:")
    inputs = ["summarize: " + item for item in sample["input"]]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["output"], max_length=max_target_length, padding=padding, truncation=True)

    # Replace padding token ID with -100 to ignore during loss computation
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the dataset
print("Tokenizing dataset...")
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["input", "output"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")


Loading tokenizer...
Loading dataset...


Found cached dataset json (/home/surenoobster/.cache/huggingface/datasets/json/default-2fa289790e7b13c7/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

Calculating max input and target lengths...


Map:   0%|          | 0/4884 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/4884 [00:00<?, ? examples/s]

Max target length: 512
Tokenizing dataset...


Map:   0%|          | 0/4884 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']
